In [1]:
import os.path as osp
from tqdm.auto import tqdm
import numpy as np
import wandb

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv, VGAE, GatedGraphConv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import DataLoader

from dataset_processing import RNADataset


/home/vdshk/SecondaryStructurePredictionGNN/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = RNADataset(root="./data/")
dataset = dataset.shuffle()
train_data, val_data, test_data = dataset[0:655], dataset[655:873], dataset[873:]

train_dataloader = DataLoader(train_data, batch_size=1, shuffle=False)
val_dataloader = DataLoader(val_data, batch_size=1, shuffle=False)


In [93]:
def precision(y_pred, y_true):
    y_pred[(y_pred > 0.5)] = 1
    y_pred[(y_pred <= 0.5)] = 0 
    
    tp = torch.sum(y_pred * y_true)
    fp = torch.sum((1 - y_true) * y_pred)
    
    return tp / (tp + fp + epsilon)

def recall(y_pred, y_true):
    y_pred[(y_pred > 0.5)] = 1
    y_pred[(y_pred <= 0.5)] = 0
    
    tp = torch.sum(y_pred * y_true)
    fn = torch.sum(y_true * (1 - y_pred))
    
    return tp / (tp + fn + epsilon)

def f1_loss(y_pred, y_true):
    tp = torch.sum(y_pred * y_true)
    fn = torch.sum(y_true * (1 - y_pred))
    fp = torch.sum((1 - y_true) * y_pred)
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)

#     k1 = 1 - torch.abs(precision - recall)
#     k2 = 1 - torch.abs(K.mean(precision) - K.mean(recall))
    #calculate upgraded f1 score
    f1 = 2 * precision * recall / (precision + recall + epsilon)
#     tw = K.sum(K.cast(y_true * y_pred, ’float32’), axis=[1, 2, 3])
#     fw = K.sum(K.cast((1 - y_true) * y_pred, ’float32’), axis=[1, 2, 3])
#     fb = K.sum(K.cast(y_true * (1 - y_pred), ’float32’), axis=[1, 2, 3])
    return 1 - f1


In [112]:
class GatedGCNModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, num_layers, out_channels, dropout):
        super(GatedGCNModel, self).__init__()
        self.ggcn = GatedGraphConv(out_channels, num_layers)

    def forward(self, x, edge_index):
        x = self.ggcn(x, edge_index)
        prob_adj = (x @ x.t()).sigmoid()
#         return (prob_adj > 0).nonzero(as_tuple=False).t()
        return prob_adj


In [249]:
hidden_channels = 128
out_channels = 300
num_layers = 6
num_features = dataset.num_features


dr = 0.2
lr = 0.00007
epochs = 580

model = GatedGCNModel(num_features, hidden_channels, num_layers, out_channels, dr)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
train_data = list(map(lambda x: x.to(device), train_data))
val_data = list(map(lambda x: x.to(device), val_data))
print(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# optimizer = torch.optim.Adagrad(model.parameters(), lr=lr)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
# criterion = torch.nn.BCEWithLogitsLoss()

# def RMSELoss(y_pred, y_true):
#     return torch.sqrt(torch.mean((y_pred - y_true) ** 2))

criterion = f1_loss

cuda


In [250]:
run_name = "GatedGCN_" + str(num_layers) + "_" + str(epochs) + "_" + str(lr) + "_" + "Adam_" + str(out_channels)


wandb.init(
    # set the wandb project where this run will be logged
    project="secondary_structure_prediction",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "GatedGCN",
    "epochs": epochs,
    "optimizer": "Amsgrad",
    "out_channels": out_channels,
    "loss": "f1_loss",
    "train:val:test": "655:218:218"
    },
    name=run_name
)

epsilon = 1e-10

train_f1,▁▃▄▃▅▆▆▇█
train_loss,█▆▅▅▄▃▃▂▁
train_precision,▁▃▄▄▅▆▆▇█
train_recall,█▃▁▁▁▁▅▃▆
val_f1,▁▄▄▄▆▆▆▇█
val_loss,█▅▄▅▃▃▂▂▁
val_precision,▁▄▅▄▆▆▆▇█
val_recall,█▁▃▆▂▅▅▅▆
train_f1,0.19933
train_loss,0.81098
train_precision,0.11251


In [251]:
def train():
    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = []
        train_recall = []
        train_precision = []
        for g in tqdm(train_data, ncols=100):
            g.to(device)
            optimizer.zero_grad()

            out = model(g.x, g.edge_index)
            y_true = g.edge_label_index
            loss = criterion(out, y_true)            
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            train_precision.append(precision(out, y_true).item())
            train_recall.append(recall(out, y_true).item())
            
        train_prec = np.mean(train_precision)
        train_rec = np.mean(train_recall)
        train_f1 = (2 * train_prec * train_rec) / (train_prec + train_rec)
        print(f'Epoch: {epoch:03d}, loss: {np.mean(train_loss)}, f1: {train_f1}, precision: {train_prec}, recall: {train_rec}')
        
        val_loss = []
        val_recall = []
        val_precision = []
        with torch.no_grad():
            for g in tqdm(val_data, ncols=100):
                g.to(device)
                out = model(g.x, g.edge_index)
        
                y_true = g.edge_label_index
                loss = criterion(out, y_true)
                                
                val_loss.append(loss.item())
                val_precision.append(precision(out, y_true).item())
                val_recall.append(recall(out, y_true).item())
 
            
            prec = np.mean(val_precision)
            rec = np.mean(val_recall)
            f1 = (2 * prec * rec) / (prec + rec)
            print(f'val_loss: {np.mean(val_loss)}, val_f1: {f1}, val_precision: {prec}, val_recall: {rec}')
              
            wandb.log({"train_loss": np.mean(train_loss), "train_f1": train_f1, "train_precision": train_prec, 
                       "train_recall": train_rec,
                       "val_loss": np.mean(val_loss), "val_f1": f1, "val_precision": prec, "val_recall": rec})
#     wandb.finish()

In [ ]:
train()
torch.save(model, "./models/" + run_name + ".pt")

100%|████████████████████████████████████████████████████████████| 655/655 [00:04<00:00, 154.10it/s]


Epoch: 001, loss: 0.8652826594942399, f1: 0.14060851103907063, precision: 0.07632150753251923, recall: 0.8917237894680664


100%|████████████████████████████████████████████████████████████| 218/218 [00:00<00:00, 502.94it/s]


val_loss: 0.8570214072498706, val_f1: 0.14793691111080093, val_precision: 0.081119082825378, val_recall: 0.8391226456252807


100%|████████████████████████████████████████████████████████████| 655/655 [00:04<00:00, 151.44it/s]


...

100%|████████████████████████████████████████████████████████████| 655/655 [00:04<00:00, 150.76it/s]


Epoch: 572, loss: 0.6965668641883909, f1: 0.32175033333110004, precision: 0.19671264862684348, recall: 0.8830469776655882


100%|████████████████████████████████████████████████████████████| 218/218 [00:00<00:00, 516.61it/s]


val_loss: 0.7147266049997523, val_f1: 0.30082989153558165, val_precision: 0.1812350029518845, val_recall: 0.8845036521964117


 34%|████████████████████▌                                       | 224/655 [00:01<00:02, 150.37it/s]

In [59]:
run_name = "GatedGCN_" + str(num_layers) + "_" + "140" + "_" + str(lr) + "_" + "Adam_" + str(out_channels)


torch.save(model, "./models/" + run_name + ".pt")